## 使用多层 layer

加载必要的库并重置计算图

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.framework import ops
ops.reset_default_graph()

### 创建图会话

In [2]:
sess = tf.Session()

### 创建 tensor

这里我们创建一个 4x4 像素的图像并通过多层 layer 传播它

In [3]:
# 创建一个随机 4x4 图像
x_shape = [1, 4, 4, 1]
x_val = np.random.uniform(size=x_shape)

### 创建数据 PlaceHolder

In [4]:
x_data = tf.placeholder(tf.float32, shape=x_shape)

### 第一层：移动窗口（卷积）

我们的第一层 layer 是一个大小为 2x2 步幅为 2 的（在高度和宽度方向上）空间移动窗口。

为了使其成为移动窗口平均值，滤波器的值将全部为0.25。

In [5]:
# 创建一个采用空间移动窗口平均的层。
# 我们的窗口将为2x2，高度和宽度均为2。
# 过滤器值将为0.25，因为我们需要2x2窗口的平均值
my_filter = tf.constant(0.25, shape=[2, 2, 1, 1])
my_strides = [1, 2, 2, 1]
mov_avg_layer = tf.nn.conv2d(x_data, my_filter, my_strides,
                            padding='SAME', name='Moving_Avg_Window')


### 第二层：自定义

我们的第二层是自定义层。给个输入，这层平坦化 x，并计算 sigmoid(Ax + b)。这里，A 和 b 将是预定的常数。

然后我们天家自定义层到图并命名“Custom_Layer”。稍后会在 tensorboard 中可视化。

In [6]:
# 定义自定义层来计算 sigmoid(Ax + b)，x 是个 2x2 矩阵，A 和 b 也是 2x2 矩阵
def custom_layer(input_matrix):
    input_matrix_sqeezed = tf.squeeze(input_matrix)
    A = tf.constant([[1., 2.], [-1., 3.]])
    b = tf.constant(1., shape=[2, 2])
    temp1 = tf.matmul(A, input_matrix_sqeezed)
    temp = tf.add(temp1, b) # Ax + b
    return tf.sigmoid(temp)

# 添加自定义层到计算图
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_layer)

### 运行并输出

输出会是个 2x2 的数组，但 size 为 (1, 2, 2, 1)

In [7]:
print(sess.run(mov_avg_layer, feed_dict={x_data: x_val}))

[[[[ 0.64304769]
   [ 0.44053   ]]

  [[ 0.64040095]
   [ 0.57584316]]]]


自定义操作后，size 为 2x2（squeeze 后变成一维），如下：

In [8]:
print(sess.run(custom_layer1, feed_dict={x_data: x_val}))

[[ 0.94901294  0.93035901]
 [ 0.90705168  0.90779465]]


保存概要以可视化：

In [9]:
merged = tf.summary.merge_all(key='summaries')

if not os.path.exists('tensorboard_logs/'):
    os.mkdir('tensorboard_logs/')

my_writer = tf.summary.FileWriter('tensorboard_logs/', sess.graph)

![png](./03.png)